# Imports 

In [5]:
import numpy as np, pandas as pd 
pd.set_option('display.max_columns', None)
import sqlalchemy, sys, os  

sys.path.append(os.path.abspath(os.path.join(os.path.dirname('functions.py'), '..', '..', 'PY')))
import functions  

seasons = functions.generate_seasons(1949, 2024)

# Output Tables (Multiple Seasons) 

In [9]:
engine = sqlalchemy.create_engine("sqlite:///../../DB/ballbase.db") 
pd.read_sql('master_1973_74', con=engine).sort_values('PPG', ascending=False)  

,Player,Age,Team,Pos,GP,MPG,FG,FGA,FG%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORPG,DRPG,RPG,APG,SPG,BPG,PF,PPG,Awards,T2P,T2PA,TAST,TBLK,TDRB,TFG,TFGA,TFT,TFTA,TMP,TORB,TPF,TPTS,TSTL,TTRB,DRtg,GS,AST%,BLK%,BPM,DBPM,DRB%,DWS,FTr,OBPM,ORB%,OWS,PER,STL%,TRB%,TS%,VORP,WS,WS/48
17,Bob McAdoo,22.0,BUF,C,74.0,43.0,12.2,22.3,54.7,12.2,22.3,54.7,54.7,6.2,7.8,79.3,3.8,11.3,15.1,2.3,1.2,3.3,3.4,30.6,"MVP-2,AS,NBA2",901.0,1647.0,170.0,246.0,836.0,901.0,1647.0,459.0,579.0,3185.0,281.0,252.0,2261.0,88.0,1117.0,95.0,NaN,8.1,3.8,5.0,0.3,25.4,4.4,0.352,4.6,9.0,10.9,24.7,1.2,17.4,59.4,5.6,15.3,0.231
173,Pete Maravich,26.0,ATL,SG,76.0,38.2,10.8,23.6,45.7,10.8,23.6,45.7,45.7,6.2,7.5,82.6,1.3,3.6,4.9,5.2,1.5,0.2,3.4,27.7,"MVP-17,AS",819.0,1791.0,396.0,13.0,276.0,819.0,1791.0,469.0,568.0,2903.0,98.0,261.0,2107.0,111.0,374.0,101.0,NaN,21.7,0.2,1.1,-1.3,9.7,1.9,0.317,2.5,3.3,5.7,20.3,1.7,6.5,51.6,2.3,7.6,0.125
122,Kareem Abdul-Jabbar,26.0,MIL,C,81.0,43.8,11.7,21.7,53.9,11.7,21.7,53.9,53.9,3.6,5.2,70.2,3.5,11.0,14.5,4.8,1.4,3.5,2.9,27.0,"MVP-1,AS,NBA1",948.0,1759.0,386.0,283.0,891.0,948.0,1759.0,295.0,420.0,3548.0,287.0,238.0,2191.0,112.0,1178.0,89.0,NaN,16.1,4.0,7.5,2.5,23.9,7.9,0.239,5.0,8.8,10.6,24.4,1.4,16.9,56.4,8.6,18.4,0.250
28,Charlie Scott,25.0,PHO,PG,52.0,38.5,10.3,22.5,45.9,10.3,22.5,45.9,45.9,4.7,6.1,78.1,1.2,3.0,4.3,5.2,1.9,0.4,3.7,25.4,AS,538.0,1171.0,271.0,22.0,158.0,538.0,1171.0,246.0,315.0,2003.0,64.0,194.0,1322.0,99.0,222.0,102.0,NaN,21.4,0.6,0.8,-1.0,7.9,0.9,0.269,1.8,3.3,2.9,18.8,2.2,5.6,50.5,1.4,3.9,0.093
140,Lou Hudson,29.0,ATL,SF,65.0,39.8,10.4,20.9,50.0,10.4,20.9,50.0,50.0,4.5,5.4,83.6,1.9,3.4,5.4,3.3,2.5,0.4,3.2,25.4,AS,678.0,1356.0,213.0,29.0,224.0,678.0,1356.0,295.0,353.0,2588.0,126.0,205.0,1651.0,160.0,350.0,99.0,NaN,12.7,0.6,2.9,0.3,8.9,2.4,0.260,2.6,4.8,6.5,19.5,2.7,6.8,54.6,3.2,8.8,0.164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,Tom Patterson,25.0,CAP,SF,2.0,4.0,0.0,0.5,0.0,0.0,0.5,0.0,0.0,0.5,1.0,50.0,0.5,0.5,1.0,1.0,0.0,0.0,0.0,0.5,None,0.0,1.0,2.0,0.0,1.0,0.0,1.0,1.0,2.0,8.0,1.0,0.0,1.0,0.0,2.0,99.0,NaN,28.3,0.0,-4.3,0.0,12.1,0.0,2.000,-4.3,11.7,0.0,11.0,0.0,11.9,26.6,0.0,0.0,-0.056
169,Paul McCracken,23.0,HOU,SG,4.0,3.3,0.3,1.0,25.0,0.3,1.0,25.0,25.0,0.0,0.0,NaN,0.3,1.3,1.5,0.5,0.0,0.0,0.8,0.5,None,1.0,4.0,2.0,0.0,5.0,1.0,4.0,0.0,0.0,13.0,1.0,3.0,2.0,0.0,6.0,97.0,NaN,18.7,0.0,-4.9,-0.1,40.7,0.0,0.000,-4.8,8.3,-0.1,6.1,0.0,24.7,25.0,0.0,0.0,-0.152
99,Jim Garvin,23.0,BUF,PF,6.0,1.8,0.2,0.7,25.0,0.2,0.7,25.0,25.0,0.0,0.0,NaN,0.2,0.7,0.8,0.0,0.0,0.0,0.2,0.3,None,1.0,4.0,0.0,0.0,4.0,1.0,4.0,0.0,0.0,11.0,1.0,1.0,2.0,0.0,5.0,98.0,NaN,0.0,0.0,-5.0,-0.3,35.2,0.0,0.000,-4.7,9.3,0.0,3.9,0.0,22.6,25.0,0.0,0.0,-0.134
49,Dennis Bell,22.0,NYK,SF,1.0,4.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,103.0,0.0,0.0,0.0,-4.8,-0.1,0.0,0.0,0.000,-4.7,0.0,0.0,-6.8,0.0,0.0,0.0,0.0,0.0,-0.222


In [18]:
engine = sqlalchemy.create_engine("sqlite:///../../DB/ballbase.db") 
for season in seasons:
    season_str = season 
    master = pd.read_sql("master_"+season, con=engine).fillna(0) 
    if '3P' in master.columns: 
        averages = master[['Player', 'Pos', 'Team', 'Age', 'PPG', 'RPG', 'APG', 'SPG', 'BPG', 'ToPG', 'MPG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P%', 'FTA', 'FT%', 'PF']] 
        totals = master.loc[:, ['Player', 'TPTS', 'TSTL',
            'TTOV', 'TTRB', 'T2P', 'T2PA', 'T3P', 'T3PA', 'TAST', 'TBLK', 'TDRB',
            'TFG', 'TFGA', 'TFT', 'TFTA', 'TMP', 'TORB', 'TPF']]
        advanced = master.loc[:, ['Player', 'DRtg', 'ORtg',
            '3PAr', 'AST%', 'BLK%', 'BPM', 'DBPM', 'DRB%', 'DWS', 'FTr', 'OBPM',
            'ORB%', 'OWS', 'PER', 'STL%', 'TOV%', 'TRB%', 'TS%', 'eFG%', 'USG%', 'VORP',
            'WS', 'WS/48']] 
    elif 'BPG' in master.columns:  
        averages = master[['Player', 'Pos', 'Team', 'Age', 'PPG', 'RPG', 'APG', 'SPG', 'BPG', 'MPG', 'FGA', 'FG%', '2P%', 'FTA', 'FT%', 'PF']] 
        totals = master.loc[:, ['Player', 'Pos', 'Team', 'Age', 'TPTS', 'TSTL', 'TTRB', 'T2P', 'T2PA', 'TAST', 'TBLK', 'TDRB', 'TFG', 'TFGA', 'TFT', 'TFTA', 'TMP', 'TORB', 'TPF']]
        advanced = master.loc[:, ['Player', 'Pos', 'Team', 'Age', 'DRtg', 'AST%', 'BLK%', 'BPM', 'DBPM', 'DRB%', 'DWS', 'FTr', 'OBPM',
            'ORB%', 'OWS', 'PER', 'STL%', 'TRB%', 'TS%', 'eFG%', 'VORP',
            'WS', 'WS/48']] 
    elif 'RPG' in master.columns: 
        averages = master[['Player', 'Pos', 'Team', 'Age', 'PPG', 'RPG', 'APG', 'GP', 'FGA', 'FG%', '2P%', 'FTA', 'FT%', 'PF']]  
        totals = master.loc[:, ['Player', 'Pos', 'Team', 'Age', 'TPTS', 'TAST', 'TTRB', 'T2P', 'T2PA', 'TFG', 'TFGA', 'TFT', 'TFTA', 'TPF']] 
        advanced = master.loc[:, ['Player', 'Pos', 'Team', 'Age', 'DWS', 'FTr', 'OWS', 'TS%', 'eFG%', 'WS']]   
    else: 
        averages = master[['Player', 'Pos', 'Team', 'Age', 'PPG', 'APG', 'FGA', 'FG%', '2P%', 'FTA', 'FT%', 'PF']] 
        totals = master.loc[:, ['Player', 'Pos', 'Team', 'Age', 'T2P', 'T2PA', 'TAST', 'TFG', 'TFGA', 'TFT', 'TFTA', 'TPF', 'TPTS']] 
        advanced = master.loc[:, ['Player', 'Pos', 'Team', 'Age', 'DWS', 'FTr', 'OWS', 'TS%', 'WS']] 


    contents = [averages, totals, advanced] 
    # Create HTML table version 
    for i in range(len(contents)):
        contents[i] = contents[i].to_html(table_id="table", classes = "table table-hover table-stripped table-bordered table-striped", border=1, index=False)
        contents[i] = contents[i].replace('<table border="1" class="dataframe table table-hover table-stripped table-bordered table-striped" id="table">', '<table border="1" class="dataframe table table-hover table-stripped table-bordered" style="width:100%" id="table">')
    # Create page content 
    for i in range(len(contents)):
        contents[i] = f"""
{{%extends 'TableBase.html'%}}
{{% block table %}} 

{contents[i]}
{{% endblock %}}
"""
    def write_to_file(path, content): 
        file = open(path, "w", encoding="utf-8") 
        file.write(content) 
        file.close 


    file_names=[f"Averages_{season}.html", f"Totals_{season}.html", f"Advanced_{season}.html"] 

    for i in range(len(contents)): 
        path = f"../../templates/Tables/{file_names[i]}" 
        write_to_file(path, contents[i]) 

# Output Season Buttons 

In [19]:
season_labs = functions.generate_seasons(1949, 2024)  
temp = []

season_button_content = ""
for season in season_labs: 
    ssn_str = season.replace('_', '-') 
    season_button_content += f'<button onclick="seasonSelect(\'{season}\')" class="dropdown-button"> {ssn_str} </button>' + '\n'  
    temp.append(f'<button onclick="seasonSelect(\'{season}\')" class="dropdown-button"> {ssn_str} </button>') 
for i in range(len(temp)-1, -1, -1):
    print(temp[i]) 
# file_b = open("../../templates/Other/Season-buttons.html", "w", encoding="utf-8") 
# file_b.write(season_button_content) 
# file_b.close()  
# print(season_button_content)

<button onclick="seasonSelect('2023_24')" class="dropdown-button"> 2023-24 </button>
<button onclick="seasonSelect('2022_23')" class="dropdown-button"> 2022-23 </button>
<button onclick="seasonSelect('2021_22')" class="dropdown-button"> 2021-22 </button>
<button onclick="seasonSelect('2020_21')" class="dropdown-button"> 2020-21 </button>
<button onclick="seasonSelect('2019_20')" class="dropdown-button"> 2019-20 </button>
<button onclick="seasonSelect('2018_19')" class="dropdown-button"> 2018-19 </button>
<button onclick="seasonSelect('2017_18')" class="dropdown-button"> 2017-18 </button>
<button onclick="seasonSelect('2016_17')" class="dropdown-button"> 2016-17 </button>
<button onclick="seasonSelect('2015_16')" class="dropdown-button"> 2015-16 </button>
<button onclick="seasonSelect('2014_15')" class="dropdown-button"> 2014-15 </button>
<button onclick="seasonSelect('2013_14')" class="dropdown-button"> 2013-14 </button>
<button onclick="seasonSelect('2012_13')" class="dropdown-button"